In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import os
import asf_search as asf
from collections import defaultdict
import datetime
import pickle
from tqdm import tqdm
from functools import reduce
import itertools
from shapely import Point, LineString, Polygon, MultiPolygon
import operator
import lxml
from pykml.factory import KML_ElementMaker as KML
import cartopy

In [2]:
def valid_data_extraction(ds, time_filter, colum_names):
    #Make sure colum_names is a list
    if type(colum_names) is str:
        colum_names = [colum_names]
        
    #Add longitude, latidude and position_qc as variables indexed by time,depth as all other variables
    TIME = ds['TIME'].values
    DEPTH = ds['DEPTH'].values
    n_DEPTHS = len(DEPTH)

    dataset_columns = {
        'LONG':ds['LONGITUDE'],
        'LAT':ds['LATITUDE'],
        'POS_QC':ds['POSITION_QC'],
    }

    ds_pos = xr.Dataset(
        data_vars=
        {k:(
            ["TIME", 'DEPTH'],
            np.repeat(np.reshape(v.values, (-1,1)), n_DEPTHS, axis=1),
            v.attrs,
        )for (k,v) in dataset_columns.items()},
        coords=dict(
            TIME=TIME,
            DEPTH=DEPTH,
        )
    ).drop_vars('DEPTH')
    ds = xr.merge([ds.drop_dims(['LATITUDE', 'LONGITUDE', 'POSITION']), ds_pos])
    
    #Filter for time of interest and depth 0
    if type(time_filter) is tuple:
        ds = ds.sel(TIME=slice(time_filter[0], time_filter[1]), DEPTH=0)
    else:
        ds = ds.sel(TIME=time_filter, DEPTH=0)   
    
    #Filter only avalible columns
    colum_names = list(set(colum_names).intersection(list(ds.data_vars)))
    colum_names_qc = [c + '_QC' for c in colum_names]
    
    #Add fixed columns
    colum_names.extend(['LONG', 'LAT'])
    colum_names_qc.extend(['TIME_QC', 'POS_QC'])
    
    #Filter for columns of interest
    ds = ds[colum_names + colum_names_qc]
    
    df = ds.to_dataframe()
    QC_good = [1.0, 7.0]
    
    #Filter for good data only
    filter_qc = [df[c_qc].isin(QC_good) for c_qc in colum_names_qc]
    #Element-wise AND the filters
    filter_qc = reduce(operator.and_, filter_qc)
    df = df[filter_qc][colum_names]
    
    return df

In [3]:
# Searches fro SAR images that overlaps with the data in file according to the filters:
# Within the start and end date
# Only for variables found in the variables argument
# Absolute time difference between image and bouy observation is at most max_time_diff_s
# The data is also quiality controlled and only rows containing valid data for all variables are accepted
# The time and position fields are also quality controlled 
# It also excludes datapoints found within the land Multipolygon

# The result is saved in result_df wich is automatically created with the correct column datatypes after first passing None
def search_file(file, data_dir, start_date, end_date, variables, max_time_diff_s, result_df, land_multipolygon):
    #Conditionally create the result dataframe
    if result_df is None:
        result_df = pd.DataFrame({c: pd.Series(dtype=t) for c, t in {
            'bouy_file_name':str,
            'bouy_longitude':float,
            'bouy_latitude':float,
            'bouy_time':np.dtype('<M8[ns]'), #np.datetime64
            'bouy_variable_name':str,
            'bouy_variable_value':float,
            'sar_url':str,
            'sar_mode':str,
            'sar_polarization':str,
            'sar_platform':str,
            'sar_start_time':np.dtype('<M8[ns]'), #np.datetime64
            'sar_stop_time':np.dtype('<M8[ns]'), #np.datetime64
            'sar_coordinates':object, #List
        }.items()})

    #Load the data from the file
    file_path = os.path.join(data_dir, file)
    xar = xr.open_dataset(file_path)# , engine='scipy')
    xar_vars = list(xar.data_vars)

    #Clear any eventual prior asf search result
    asf_results = None
    
    #Filter for variables that exist in the data
    common_variables = set(variables).intersection(xar_vars)
    
    if len(common_variables) == 0:
        print(file + " does not have any of the variables " + str(variables))
        return result_df
    
    for var in common_variables:
        try:
            #extract dataframe of the correct timeinterval and variable 
            df = valid_data_extraction(xar, (start_date, end_date), var)
        except Exception as e:
            print(file, ': data extraction failed with:', e)
            continue
        
        if df.empty:
            print(file + " does not have any valid data for this timeperiod")
            continue
            
        #Only search once (assume similar results in regards to min and max longitude and latitude for each variable)
        if asf_results is None:
            #extract long and latitude
            long_min, lat_min = df[['LONG', 'LAT']].min()
            long_max, lat_max = df[['LONG', 'LAT']].max()

            #Create geographical search restriction
            coord_points = list(itertools.product(set([long_min, long_max]), set([lat_min, lat_max])))

            #For each possible length of coord_points set the prefix ans dufix for the asf search
            #Also create a shapely geometry to check for overlap with the land_multipolygon
            if len(coord_points) == 1:
                geo_prefix = 'POINT('
                geo_suffix = ')'
                geo_obj = Point(*coord_points)
            elif len(coord_points) == 2:
                #raise Exception("First time we see only changes in one coordinate, debug and make sure it works porpery")
                geo_prefix = 'LINESTRING('
                geo_suffix = ')'
                geo_obj = LineString(coord_points)
            elif len(coord_points) == 4:
                #Double parenthesis was given from asf website export search as python function
                geo_prefix = 'POLYGON(('
                geo_suffix = '))'
                #Untangle the polygon itersection
                coord_points[2:4]=reversed(coord_points[2:4])
                #Form closed polygon by adding adding fist point as last
                coord_points.append(coord_points[0])
                geo_obj = Polygon(coord_points)

            #The area that of the locations in the data that is not near the shoreline
            off_shore = geo_obj.difference(land_multipolygon)

            #Check if the whole area of interest is close to shore or overlapps with land
            #in that case return
            if off_shore.is_empty:
                print(file + " is too close to land")
                return result_df

            #Filter data of df to only include data that does not lie close to shore
            #allow a small distance since points on the edge doeas not overlap according to shapely.overlaps 
            df_shore_filter =  df.apply(lambda row: Point([row['LONG'], row['LAT']]).distance(off_shore), axis=1) <= 0.00001
            df = df[df_shore_filter]

            geo_limit = ','.join([str(long) + ' ' + str(lat) for long,lat in coord_points])
            geo_limit = geo_prefix + geo_limit + geo_suffix
            
            print(geo_limit)

            #Search asf
            options = {
                'intersectsWith': geo_limit,
                'platform': 'SENTINEL-1',
                'instrument': 'C-SAR',
                'start': start_date,
                'end': end_date,
                'processingLevel': [
                    'GRD_HD',
                    'GRD_MD',
                    'GRD_MS',
                    'GRD_HS'
                ],
                'beamSwath': [
                    'IW',
                    'EW'
                ],
            }

            asf_results = asf.search(**options)
            print(len(asf_results), ' results found')
        
        for asf_result in asf_results:
            #Find mean time of image generation
            asf_result_start = datetime.datetime.fromisoformat(asf_result.properties['startTime'][:-5])
            asf_result_stop = datetime.datetime.fromisoformat(asf_result.properties['stopTime'][:-5])
            asf_result_mean_time = asf_result_start + (asf_result_stop-asf_result_start)/2
        
            #Check the shape type
            if asf_result.geometry['type'] != 'Polygon':
                raise ValueError('The shape of the search result is not Polygon but instead: ' + asf_result.geometry['type'])
            
            #Filter for points that lie within the image
            if geo_prefix != 'POINT(':
                polygon = Polygon(asf_result.geometry['coordinates'][0])
                df_dist_filter =  df.apply(lambda row: Point([row['LONG'], row['LAT']]).distance(polygon), axis=1) <= 0.0001
                df_close = df[df_dist_filter]
                
                #Skip is we have no overlaping points
                if df_close.empty:
                    #print("No geographical overlap")
                    continue
                    
            #No geographical overlap check needed (point search)
            else:
                df_close = df
            
            #Find closest entry in time
            closest_data_row = df_close.iloc[df_close.index.get_indexer([asf_result_mean_time], method='nearest')] #only one row of data

            max_time_diff = datetime.timedelta(seconds=max_time_diff_s)
            
            #Check if it is close enough to be considered overlapping
            if abs(closest_data_row.index[0] - asf_result_mean_time) <= max_time_diff:
                #Save result in the dataframe
                result_df.loc[len(result_df.index)] = [
                    file, #'bouy_file_name':str,
                    closest_data_row['LONG'][0], #'bouy_longitude':float,
                    closest_data_row['LAT'][0], #'bouy_latitude':float,
                    closest_data_row.index[0], #'bouy_time':np.dtype('<M8[ns]'), #np.datetime64
                    var, #'bouy_variable_name':str,
                    closest_data_row[var][0], #'bouy_variable_value':float,
                    asf_result.properties['url'], #'sar_url':str,
                    asf_result.properties['beamModeType'], #'sar_mode':str,
                    asf_result.properties['polarization'].split('+'), #'sar_polarization':str,
                    asf_result.properties['platform'], #'sar_platform':str,
                    asf_result_start, #'sar_start_time':np.dtype('<M8[ns]'), #np.datetime64
                    asf_result_stop, #'sar_stop_time':np.dtype('<M8[ns]'), #np.datetime64
                    asf_result.geometry['coordinates'][0], #'sar_coordinates':list,
                ]
                
            else:
                #print('No time overlap')
                pass
            
        
    return result_df

In [4]:
data_dir = '/data/exjobb/sarssw/bouy/INSITU_GLO_PHYBGCWAV_DISCRETE_MYNRT_013_030/MO'
start_date = '2021-01-01'
end_date = '2021-12-31'
variables = ['VHM0', 'VAVH', 'WSPD']#['VHM0', 'VAVH', 'WSPD']
max_time_diff_s = 60*60

write_folder = './1h_survey'
result_df_fn = 'result_df'
kml_pinmap_fn = 'kml_pinmap'

#Dataframe where the results is saved, automatically created in search_file
result_df = None

#Load and create land multipolygon, buffered (expanded) to limit distance to shore
land_list = list(cartopy.feature.NaturalEarthFeature('physical', 'land', '50m').geometries())
polygon_list = []
for p  in land_list:
    if type(p) == MultiPolygon:
        polygon_list.extend(p.geoms)
    else:
        polygon_list.append(p)
land_multipolygon = MultiPolygon([p.buffer(0.01) for p in polygon_list])

# Create search_file function with only necessary parameters for the file loop below
def search_file_fixed_params(file, result_df):
    return search_file(file, data_dir, start_date, end_date, variables, max_time_diff_s, result_df, land_multipolygon)

files = os.listdir(data_dir)
#print(files)
#TODO reset to all files
"""
files = ['GL_TS_MO_23014.nc', 'GL_TS_MO_13010.nc',
       'IR_TS_MO_Tenerife-coast-buoy.nc', 'GL_TS_MO_23009.nc',
       'IR_TS_MO_Malaga-coast-buoy.nc', 'GL_TS_MO_55018.nc',
       'GL_TS_MO_55036.nc', 'GL_TS_MO_1500008.nc', 'GL_TS_MO_4100300.nc',
       'GL_TS_MO_55020.nc', 'MO_TS_MO_NADR-S1.nc', 'GL_TS_MO_55032.nc',
       'GL_TS_MO_55029.nc', 'BO_TS_MO_Norrbyn.nc', 'GL_TS_MO_55022.nc',
       'GL_TS_MO_55033.nc', 'GL_TS_MO_55046.nc', 'GL_TS_MO_55017.nc',
       'GL_TS_MO_41121.nc', 'GL_TS_MO_31004.nc', 'GL_TS_MO_55027.nc',
       'GL_TS_MO_55045.nc', 'GL_TS_MO_1500009.nc', 'GL_TS_MO_2300453.nc',
       'GL_TS_MO_55028.nc', 'GL_TS_MO_55037.nc', 'GL_TS_MO_55030.nc',
       'GL_TS_MO_23016.nc', 'GL_TS_MO_53040.nc', 'GL_TS_MO_14041.nc',
       'GL_TS_MO_2300452.nc', 'GL_TS_MO_2300091.nc', 'GL_TS_MO_23013.nc',
       'GL_TS_MO_55038.nc', 'GL_TS_MO_2300454.nc', 'GL_TS_MO_56005.nc',
       'GL_TS_MO_31003.nc', 'GL_TS_MO_4800400.nc', 'GL_TS_MO_52121.nc',
       'GL_TS_MO_2300451.nc', 'GL_TS_MO_31005.nc', 'GL_TS_MO_52087.nc',
       'GL_TS_MO_15002.nc', 'GL_TS_MO_2300497.nc', 'GL_TS_MO_55024.nc']
"""
#files = ['BO_TS_MO_Norrbyn.nc']
file_filter = [
    'GL_TS_MO_41121.nc', #Flips longitude sign in the middle of the data, from 66 to -66???! resutlts in asf search with over 7000 matches.
]

for file in tqdm(set(files).difference(file_filter)):
    print('\n',file)
    result_df = search_file_fixed_params(file, result_df)

#Conditionally creates the folder for the result
os.makedirs(write_folder, exist_ok=True)

with open(os.path.join(write_folder, result_df_fn),'wb') as f_w:
    pickle.dump(result_df,f_w)


  0%|          | 1/2487 [00:00<04:34,  9.06it/s]


 NO_TS_MO_WeymouthMET.nc
NO_TS_MO_WeymouthMET.nc does not have any valid data for this timeperiod

 GL_TS_MO_41056.nc
GL_TS_MO_41056.nc does not have any valid data for this timeperiod
POINT(-65.46399688720703 18.26099967956543)
60  results found


  0%|          | 2/2487 [00:16<6:48:55,  9.87s/it]


 GL_TS_MO_MEDS171.nc
GL_TS_MO_MEDS171.nc does not have any valid data for this timeperiod

 GL_TS_MO_42023.nc
POINT(-83.08599853515625 26.010000228881836)


  0%|          | 4/2487 [00:29<5:19:56,  7.73s/it]

...

100%|█████████▉| 2477/2487 [3:24:57<00:38,  3.81s/it]


 GL_WS_MO_46259.nc
GL_WS_MO_46259.nc does not have any of the variables ['VHM0', 'VAVH', 'WSPD']

 GL_TS_MO_32487.nc
GL_TS_MO_32487.nc does not have any valid data for this timeperiod

 NO_TS_MO_FolkestoneMET.nc
NO_TS_MO_FolkestoneMET.nc does not have any valid data for this timeperiod

 GL_TS_MO_44176.nc
GL_TS_MO_44176.nc does not have any valid data for this timeperiod

 GL_TS_MO_44092.nc
GL_TS_MO_44092.nc does not have any valid data for this timeperiod

 GL_WS_MO_46213.nc
GL_WS_MO_46213.nc does not have any of the variables ['VHM0', 'VAVH', 'WSPD']

 GL_WS_MO_45007.nc
GL_WS_MO_45007.nc does not have any of the variables ['VHM0', 'VAVH', 'WSPD']

 BO_TS_MO_FinskaEnskar.nc
BO_TS_MO_FinskaEnskar.nc does not have any valid data for this timeperiod

 GL_WS_MO_41048.nc


100%|█████████▉| 2486/2487 [3:24:57<00:01,  1.64s/it]

GL_WS_MO_41048.nc does not have any of the variables ['VHM0', 'VAVH', 'WSPD']

 AR_TS_MO_Toke-Brygge-30-81.nc
AR_TS_MO_Toke-Brygge-30-81.nc does not have any of the variables ['VHM0', 'VAVH', 'WSPD']

 BS_TS_MO_EUXRo02.nc
BS_TS_MO_EUXRo02.nc does not have any valid data for this timeperiod

 AR_TS_MO_BSO3.nc
AR_TS_MO_BSO3.nc does not have any of the variables ['VHM0', 'VAVH', 'WSPD']

 GL_TS_MO_CampbellIsland.nc
GL_TS_MO_CampbellIsland.nc does not have any valid data for this timeperiod
GL_TS_MO_CampbellIsland.nc does not have any valid data for this timeperiod

 NO_TS_MO_Europlatform2.nc
POINT(3.2763888835906982 51.99861145019531)
232  results found


100%|██████████| 2487/2487 [3:25:17<00:00,  4.95s/it]


In [5]:
result_df

,bouy_file_name,bouy_longitude,bouy_latitude,bouy_time,bouy_variable_name,bouy_variable_value,sar_url,sar_mode,sar_polarization,sar_platform,sar_start_time,sar_stop_time,sar_coordinates
0,GL_TS_MO_41056.nc,-65.463997,18.261000,2021-12-29 10:09:59.999999744,WSPD,6.799,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-12-29 10:14:46,2021-12-29 10:15:11,"[[-64.408401, 17.459696], [-64.094124, 18.9706..."
1,GL_TS_MO_41056.nc,-65.463997,18.261000,2021-12-19 22:30:00.000000000,WSPD,7.799,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-12-19 22:28:35,2021-12-19 22:29:00,"[[-66.193626, 18.813972], [-65.879509, 17.3031..."
2,GL_TS_MO_41056.nc,-65.463997,18.261000,2021-12-17 10:09:59.999999744,WSPD,7.299,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-12-17 10:14:47,2021-12-17 10:15:12,"[[-64.40966, 17.459682], [-64.095413, 18.97060..."
3,GL_TS_MO_41056.nc,-65.463997,18.261000,2021-12-07 22:30:00.000000000,WSPD,7.399,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-12-07 22:28:35,2021-12-07 22:29:00,"[[-66.194122, 18.814003], [-65.879982, 17.3031..."
4,GL_TS_MO_41056.nc,-65.463997,18.261000,2021-12-05 10:09:59.999999744,WSPD,6.299,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-12-05 10:14:47,2021-12-05 10:15:12,"[[-64.409782, 17.459616], [-64.095543, 18.9705..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67648,NO_TS_MO_Europlatform2.nc,3.276389,51.998611,2021-01-08 06:10:00.000000000,VAVH,1.470,https://datapool.asf.alaska.edu/GRD_HD/SB/S1B_...,IW,"[VV, VH]",Sentinel-1B,2021-01-08 06:05:54,2021-01-08 06:06:19,"[[3.625535, 51.564022], [4.13135, 53.056099], ..."
67649,NO_TS_MO_Europlatform2.nc,3.276389,51.998611,2021-01-06 17:30:00.000000000,VAVH,1.280,https://datapool.asf.alaska.edu/GRD_HD/SB/S1B_...,IW,"[VV, VH]",Sentinel-1B,2021-01-06 17:32:29,2021-01-06 17:32:54,"[[1.747713, 51.907463], [2.236964, 50.414387],..."
67650,NO_TS_MO_Europlatform2.nc,3.276389,51.998611,2021-01-05 17:39:59.999999744,VAVH,1.820,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-01-05 17:41:27,2021-01-05 17:41:52,"[[-0.343658, 52.013187], [0.143647, 50.519848]..."
67651,NO_TS_MO_Europlatform2.nc,3.276389,51.998611,2021-01-03 06:00:00.000000000,VAVH,0.820,https://datapool.asf.alaska.edu/GRD_HD/SB/S1B_...,IW,"[VV, VH]",Sentinel-1B,2021-01-03 05:58:01,2021-01-03 05:58:26,"[[5.282145, 50.341362], [5.773809, 51.834118],..."


In [6]:
#Save KML pin maps, one for each variable

#Total number of datapoints per variable type
var_total = result_df['bouy_variable_name'].value_counts()
 
#pandas series for count of unique variable name, file name pairs
var_file_count = list(result_df.value_counts(subset=['bouy_variable_name', 'bouy_file_name']).items())

#dataframe for coordinates of unique variable name, file name pairs
var_file_coord = (result_df.groupby(['bouy_variable_name', 'bouy_file_name'])[['bouy_longitude', 'bouy_latitude']].first())


#Create separate kml maps for each variable name
KML_fldrs = {}
for var_name in np.unique(result_df['bouy_variable_name']):
    KML_fldrs[var_name] = KML.Folder(
        KML.name(var_name + " " + write_folder[write_folder.find('/')+1:]),
        KML.description('Total datapoints: ' + str(var_total[var_name])),
    )

#iterate over all unique variable name, file name pairs
for ((var_name, file_name), count) in var_file_count:
    #extract longitude and latitude
    long, lat = var_file_coord.loc[(var_name, file_name)][['bouy_longitude', 'bouy_latitude']]
    
    #Create the pin
    pin = KML.Placemark(
        KML.name(str(count)),
        KML.description(file_name),
        KML.Point(
            KML.coordinates(str(long) + "," + str(lat))
        )
    )
    KML_fldrs[var_name].append(pin)
    
for var_name, KML_fld in KML_fldrs.items():
    with open(os.path.join(write_folder, kml_pinmap_fn + '_' + var_name + '.kml'), 'w') as f_w:
        f_w.write(lxml.etree.tostring(KML_fld, pretty_print=True).decode())

In [7]:
#Save KML pin map if we have all the variables ['VHM0', 'VAVH', 'WSPD']
if all([v in variables for v in ['VHM0', 'VAVH', 'WSPD']]):
    #Number of  overlaps
    mult_var = result_df.groupby(['sar_url', 'bouy_file_name', 'bouy_longitude', 'bouy_latitude'])[['bouy_variable_name']].aggregate(lambda tdf: tdf.unique().tolist())
    mult_var['len'] = mult_var['bouy_variable_name'].apply(len)
    mult_var['contains_wind'] = mult_var['bouy_variable_name'].apply(lambda x: 'WSPD' in x)
    mult_var['also_VAVH'] = mult_var['bouy_variable_name'].apply(lambda x: 'VAVH' in x)
    mult_var['also_VHM0'] = mult_var['bouy_variable_name'].apply(lambda x: 'VHM0' in x)
    mult_var = mult_var[(mult_var['len'] > 1) & mult_var['contains_wind']].drop(labels='contains_wind', axis=1)

    wind_and_VAVH = mult_var[mult_var['also_VAVH']].drop(labels=['also_VAVH', 'also_VHM0'], axis=1).groupby(['bouy_file_name', 'bouy_longitude', 'bouy_latitude'])['len'].count()
    wind_and_VHM0 = mult_var[mult_var['also_VHM0']].drop(labels=['also_VAVH', 'also_VHM0'], axis=1).groupby(['bouy_file_name', 'bouy_longitude', 'bouy_latitude'])['len'].count()

    #Save overlapping wind and wave info to KML maps
    for kml_pinmap_fn, data_var in [('WSPD_and_VHM0', wind_and_VHM0), ('WSPD_and_VAVH', wind_and_VAVH)]:
        KML_fldr = KML.Folder(
            KML.name(kml_pinmap_fn + " " + write_folder[write_folder.find('/')+1:]),
            KML.description('Total datapoints: ' + str(data_var.sum())),
        )

        #iterate over all unique variable name, file name pairs
        for ((file_name, long, lat), count) in data_var.items():
            #Create the pin
            pin = KML.Placemark(
                KML.name(str(count)),
                KML.description(file_name),
                KML.Point(
                    KML.coordinates(str(long) + "," + str(lat))
                )
            )
            KML_fldr.append(pin)

        with open(os.path.join(write_folder, kml_pinmap_fn + '_' + var_name + '.kml'), 'w') as f_w:
            f_w.write(lxml.etree.tostring(KML_fldr, pretty_print=True).decode())